In [ ]:
import os
import textgrid
from tqdm.notebook import tqdm

# 起始当前脚本所在的路径
script_dir = os.getcwd()
textgrid_dir = os.path.join(script_dir, 'TextGrid2')    #TextGrid路径
out_dir = os.path.join(script_dir, 'processed')         #输出路径
dict_file = os.path.join(script_dir, 'rules.txt')       #替换词典路径

# 开始时清空输出路径下文件
if os.path.exists(out_dir):
    file_names = os.listdir(out_dir)
    for file_name in file_names:
        file_path = os.path.join(out_dir, file_name)
        os.remove(file_path)
else:
    os.makedirs(out_dir)
    
# 读取词典转换规则
replace_rules = {}
with open(dict_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            src, dest = line.split('\t')
            replace_rules[src] = dest
print(replace_rules)

# 统计转换次数
consonant_count = {consonant: 0 for consonant in replace_rules.keys()}
# 文件总数
total_files = len([filename for filename in os.listdir(textgrid_dir) if filename.endswith('.TextGrid')])

# 遍历 TextGrid路径的所有 .TextGrid 文件
for filename in tqdm(os.listdir(textgrid_dir), total=total_files, desc='Processing'):
    if filename.endswith('.TextGrid'):
        textgrid_file = os.path.join(textgrid_dir, filename)
        tg = textgrid.TextGrid.fromFile(textgrid_file)
        # 查找音素（phones）层
        phones_tier = None
        for tier in tg:
            if tier.name == 'phones':
                phones_tier = tier
                break
        # 替换音素标记
        for interval in phones_tier:
            for match, replace in replace_rules.items():
                if match == interval.mark:
                    consonant_count[match] += 1
                    interval.mark = replace
        # 保存修改后的 TextGrid 文件
        output_file = os.path.join(out_dir, f'{os.path.splitext(filename)[0]}.TextGrid')
        tg.write(output_file)

# 打印转换次数
print('完成如下转换次数:')
for consonant, replace in replace_rules.items():
    replace_count = consonant_count[consonant]
    print(f'{consonant}→{replace}: {replace_count}')
